In [1]:
# STEP 1: Imports
import os
import json
import pandas as pd
import boto3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
# STEP 2: Load data
df = pd.read_csv("bank.csv")
df = df.dropna()
df_encoded = pd.get_dummies(df)
X = df_encoded.drop("y_yes", axis=1, errors="ignore")
y = df_encoded.get("y_yes", df_encoded.iloc[:, -1])

In [3]:
# STEP 3: Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)

examples = X_test.copy()
examples["predicted"] = model.predict(X_test)
examples["probabilities"] = list(model.predict_proba(X_test))
wit_examples = json.loads(examples.to_json(orient="records"))

In [5]:
# STEP 5: Convert all values in wit_examples to strings for WIT compatibility
examples_str = X_test.copy()
examples_str["predicted"] = model.predict(X_test)
examples_str["probabilities"] = list(model.predict_proba(X_test))
examples_str = examples_str.astype(str)  # Convert all values to string

wit_examples = json.loads(examples_str.to_json(orient="records"))
config_builder = (
    WitConfigBuilder(wit_examples)
    .set_model_type('classification')
    .set_label_vocab(['no', 'yes'])
)
WitWidget(config_builder)

WitWidget(config={'model_type': 'classification', 'label_vocab': ['no', 'yes'], 'uses_json_input': True}, layo…

In [6]:
# STEP 5: Bedrock LLM Summary Function
def get_bedrock_llm_summary_claude(example):
    import json
    import boto3

    prompt = f"""
You are an expert financial assistant. Given this customer profile, explain in plain English why the model predicted what it did:

{json.dumps(example, indent=2)}

Summarize customer background, key factors, and prediction reasoning.
"""

    bedrock = boto3.client(
        "bedrock-runtime",
        region_name="us-east-1",
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
    )

    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3
    }

    response = bedrock.invoke_model(
        modelId="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        body=json.dumps(request_body)
    )

    result = json.loads(response["body"].read())
    return result["content"][0]["text"]

In [7]:
# STEP 6: What-If Scenarios
scenarios = [
    {"balance": "1500", "housing_yes": "0"},  # Higher balance, no housing loan
    {"duration": "600"},                         # Longer call duration
    {"pdays": "5"},                             # Had recent contact
]

for i, changes in enumerate(scenarios):
    example = wit_examples[0].copy()
    example.update(changes)
    summary = get_bedrock_llm_summary_claude(example)
    print(f"\n🔍 What-If Scenario {i+1}:")
    print(summary)


🔍 What-If Scenario 1:
# Analysis of Bank Deposit Prediction

## Customer Profile Summary
This 33-year-old married technician with tertiary education has a bank balance of $1,500. They have no default history, no personal loan, and were contacted via cellular phone in July for a 90-second call. This was their 3rd contact in the current campaign, with no previous campaign contacts.

## Key Factors Influencing the Prediction
The model predicted this customer would **not** subscribe to a term deposit, with high confidence (probability of 0%). Here's why:

1. **Limited Balance**: The $1,500 balance suggests limited disposable income for investment products.

2. **Contact Timing**: July contacts typically show lower conversion rates, possibly due to summer vacation periods when financial planning isn't a priority.

3. **Short Call Duration**: The 90-second call duration indicates minimal engagement or interest in the offering.

4. **Campaign History**: This was their 3rd contact in the curr